In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
save_figs = False

In [3]:
import os
# set to be lower priority
#os.nice(1)

In [4]:
interpolations = [
        "AE",
        "AF",
        "AG",
        "AH",
        "BE",
        "BF",
        "BG",
        "BH",
        "CE",
        "CF",
        "CG",
        "CH",
        "DE",
        "DF",
        "DG",
        "DH",
    ]

In [5]:
import pandas  as pd
from cdcp.paths import DATA_DIR, ensure_dir
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pathlib2 import Path
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
from joblib import Parallel, delayed
from datetime import datetime
from functools import reduce
import seaborn as sns
import scipy.stats

In [6]:
%matplotlib inline

In [7]:
# "2021-10-07_16-05-58_796914" '2021-10-06_13-30-08_899062' '2021-10-01_17-02-56_744915' 
statistics_timestamp = '2021-10-23_23-02-31_846685'# datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%f")#
statistics_timestamp

'2021-10-23_23-02-31_846685'

In [8]:
birds = [
    ('B1597', '2021-05-14_12-40-20_423998', 'kilosort2_5'),
    ('B1188', '2021-05-31_18-52-29_558216', 'kilosort2_5'),
    ('B1595', '2021-07-10_16-42-47_090257', 'kilosort2'),
    ('B1276', '2021-07-14_11-14-02_257025', 'kilosort2'),
    ('B1426', '2021-07-14_11-29-39_657273', 'kilosort2'),
    ('B1432', '2021-06-01_15-14-38_561892', 'kilosort2_5'),
    ('B1170', '2021-06-01_21-01-26_519005', 'kilosort2_5'),
    ('B1244', '2021-07-14_12-57-45_546674', 'kilosort2'),
    ('B1593', '2021-06-28_18-13-24_826008', 'kilosort2_5'),
    ('B1248', '2021-07-03_18-08-01_063431', 'kilosort2_5'),
    # acute
    #('B1279', '2021-04-12_16-27-07_289527', 'kilosort2_5'),
    #('B1500', '2021-08-27_09-24-48_680961', 'kilosort2_5'),
    #('B1239', '2021-05-17_22-18-51_557635', 'kilosort2'),
    #('B1459', '2021-08-26_21-21-12_755614', 'kilosort2_5'),
]
birds_df = pd.DataFrame(birds, columns = ['bird', 'timestamp', 'sorter']).set_index('bird')

In [9]:
identifiers = ['nm_by_interpolation_16_25', 'spike_count']

In [10]:
all_unit_statistics_df= []
for bird, timestamp, sorter in tqdm(birds, desc='birds'):
    
    unit_statistics_df_list = []
    for identifier in identifiers:
        unit_statistics_df = pd.read_pickle(
        DATA_DIR
        / "unit_statistics"
        / statistics_timestamp
        / identifier
        / "{}.pickle".format(bird)
    )
        unit_statistics_df_list.append(unit_statistics_df)
        
    print(bird, 'nrows per', [len(i) for i in unit_statistics_df_list])
    
    # combine dfs
    unit_statistics_df = reduce(lambda x, y: pd.merge(x, y, on = 'unit', how='outer'), unit_statistics_df_list)
    unit_statistics_df['bird'] = bird
    
    all_unit_statistics_df.append(unit_statistics_df)
unit_statistics_df = pd.concat(all_unit_statistics_df)

birds:  10%|█         | 1/10 [00:02<00:21,  2.36s/it]

B1597 nrows per [6750, 6750]


birds:  50%|█████     | 5/10 [00:02<00:01,  2.95it/s]

B1188 nrows per [510, 510]
B1595 nrows per [242, 242]
B1276 nrows per [46, 46]
B1426 nrows per [93, 93]
B1432 nrows per [2157, 2157]


birds:  70%|███████   | 7/10 [00:04<00:01,  1.93it/s]

B1170 nrows per [1281, 1281]
B1244 nrows per [141, 141]


birds:  90%|█████████ | 9/10 [00:07<00:01,  1.00s/it]

B1593 nrows per [7714, 7714]


birds: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]

B1248 nrows per [8698, 8698]


In [11]:
# merge with categorical units
categorical_units_df = pd.read_pickle(DATA_DIR / 'categorical_unit_01_23_22.pickle')
categorical_units_df= categorical_units_df[categorical_units_df.categorical_unit == True]
unit_statistics_df_cat = pd.merge(unit_statistics_df, categorical_units_df, on = ['bird', 'unit'])
# subset categorical units
#unit_statistics_df_cat = unit_statistics_df_cat[unit_statistics_df_cat.categorical_unit == True]
unit_statistics_df = unit_statistics_df_cat
print(len(unit_statistics_df))

### merge with unit types
unit_type_df = pd.read_pickle(DATA_DIR / "unit_classes_01_23_22.pickle")
#unit_type_df = unit_type_df[["nuclei","AP","ML","DV","amplitude","template_amplitude","sr","fw","hw","bird","unit"]]
unit_statistics_df_ut = pd.merge(unit_statistics_df, unit_type_df, on = ['bird', 'unit'])
unit_statistics_df = unit_statistics_df_ut
print(len(unit_statistics_df))

6754
6754


In [12]:
unit_statistics_df[:2]

,sm_correlation_AF,rv_AF,ips_AF,nm_min_correlation_AF,nm_max_correlation_AF,nm_r2_correlation_AF,nm_inflection_correlation_AF,nm_slope_correlation_AF,nm_range_correlation_AF,nm_scaled_slope_correlation_AF,sm_cosine_AF,nm_min_cosine_AF,nm_max_cosine_AF,nm_r2_cosine_AF,nm_inflection_cosine_AF,nm_slope_cosine_AF,nm_range_cosine_AF,nm_scaled_slope_cosine_AF,sm_euclidean_AF,nm_min_euclidean_AF,nm_max_euclidean_AF,nm_r2_euclidean_AF,nm_inflection_euclidean_AF,nm_slope_euclidean_AF,nm_range_euclidean_AF,nm_scaled_slope_euclidean_AF,sm_manhattan_AF,nm_min_manhattan_AF,nm_max_manhattan_AF,nm_r2_manhattan_AF,nm_inflection_manhattan_AF,nm_slope_manhattan_AF,nm_range_manhattan_AF,nm_scaled_slope_manhattan_AF,sm_correlation_AE,rv_AE,ips_AE,nm_min_correlation_AE,nm_max_correlation_AE,nm_r2_correlation_AE,nm_inflection_correlation_AE,nm_slope_correlation_AE,nm_range_correlation_AE,nm_scaled_slope_correlation_AE,sm_cosine_AE,nm_min_cosine_AE,nm_max_cosine_AE,nm_r2_cosine_AE,nm_inflection_cosine_AE,nm_slope_cosine_AE,nm_range_cosine_AE,nm_scaled_slope_cosine_AE,sm_euclidean_AE,nm_min_euclidean_AE,nm_max_euclidean_AE,nm_r2_euclidean_AE,nm_inflection_euclidean_AE,nm_slope_euclidean_AE,nm_range_euclidean_AE,nm_scaled_slope_euclidean_AE,sm_manhattan_AE,nm_min_manhattan_AE,nm_max_manhattan_AE,nm_r2_manhattan_AE,nm_inflection_manhattan_AE,nm_slope_manhattan_AE,nm_range_manhattan_AE,nm_scaled_slope_manhattan_AE,sm_correlation_BF,rv_BF,ips_BF,nm_min_correlation_BF,nm_max_correlation_BF,nm_r2_correlation_BF,nm_inflection_correlation_BF,nm_slope_correlation_BF,nm_range_correlation_BF,nm_scaled_slope_correlation_BF,sm_cosine_BF,nm_min_cosine_BF,nm_max_cosine_BF,nm_r2_cosine_BF,nm_inflection_cosine_BF,nm_slope_cosine_BF,nm_range_cosine_BF,nm_scaled_slope_cosine_BF,sm_euclidean_BF,nm_min_euclidean_BF,nm_max_euclidean_BF,nm_r2_euclidean_BF,nm_inflection_euclidean_BF,nm_slope_euclidean_BF,nm_range_euclidean_BF,nm_scaled_slope_euclidean_BF,sm_manhattan_BF,nm_min_manhattan_BF,nm_max_manhattan_BF,nm_r2_manhattan_BF,nm_inflection_manhattan_BF,nm_slope_manhattan_BF,nm_range_manhattan_BF,nm_scaled_slope_manhattan_BF,sm_correlation_BE,rv_BE,ips_BE,nm_min_correlation_BE,nm_max_correlation_BE,nm_r2_correlation_BE,nm_inflection_correlation_BE,nm_slope_correlation_BE,nm_range_correlation_BE,nm_scaled_slope_correlation_BE,sm_cosine_BE,nm_min_cosine_BE,nm_max_cosine_BE,nm_r2_cosine_BE,nm_inflection_cosine_BE,nm_slope_cosine_BE,nm_range_cosine_BE,nm_scaled_slope_cosine_BE,sm_euclidean_BE,nm_min_euclidean_BE,nm_max_euclidean_BE,nm_r2_euclidean_BE,nm_inflection_euclidean_BE,nm_slope_euclidean_BE,nm_range_euclidean_BE,nm_scaled_slope_euclidean_BE,sm_manhattan_BE,nm_min_manhattan_BE,nm_max_manhattan_BE,nm_r2_manhattan_BE,nm_inflection_manhattan_BE,nm_slope_manhattan_BE,nm_range_manhattan_BE,nm_scaled_slope_manhattan_BE,sm_correlation_CG,rv_CG,ips_CG,nm_min_correlation_CG,nm_max_correlation_CG,nm_r2_correlation_CG,nm_inflection_correlation_CG,nm_slope_correlation_CG,nm_range_correlation_CG,nm_scaled_slope_correlation_CG,sm_cosine_CG,nm_min_cosine_CG,nm_max_cosine_CG,nm_r2_cosine_CG,nm_inflection_cosine_CG,nm_slope_cosine_CG,nm_range_cosine_CG,nm_scaled_slope_cosine_CG,sm_euclidean_CG,nm_min_euclidean_CG,nm_max_euclidean_CG,nm_r2_euclidean_CG,nm_inflection_euclidean_CG,nm_slope_euclidean_CG,nm_range_euclidean_CG,nm_scaled_slope_euclidean_CG,sm_manhattan_CG,nm_min_manhattan_CG,nm_max_manhattan_CG,nm_r2_manhattan_CG,nm_inflection_manhattan_CG,nm_slope_manhattan_CG,nm_range_manhattan_CG,nm_scaled_slope_manhattan_CG,sm_correlation_AG,rv_AG,ips_AG,nm_min_correlation_AG,nm_max_correlation_AG,nm_r2_correlation_AG,nm_inflection_correlation_AG,nm_slope_correlation_AG,nm_range_correlation_AG,nm_scaled_slope_correlation_AG,sm_cosine_AG,nm_min_cosine_AG,nm_max_cosine_AG,nm_r2_cosine_AG,nm_inflection_cosine_AG,nm_slope_cosine_AG,nm_range_cosine_AG,nm_scaled_slope_cosine_AG,sm_euclidean_AG,nm_min_euclidean_AG,nm_max_euclidean_AG,nm_r2_euclidean_AG,nm_inflection_euclidean_AG,nm_slope_euclidea

### Melt matrix into 1 row per interpolation

In [13]:
interped_columns = [i  for i in unit_statistics_df.columns if i.split('_')[-1]  in interpolations]
interped_columns_unique = np.unique(['_'.join(i.split('_')[:-1]) for i in interped_columns])

In [14]:
stat_dfs = []
for col in tqdm(interped_columns_unique):
    columns_to_subset = [
        "{}_{}".format(col, i)
        for i in interpolations
        if "{}_{}".format(col, i) in unit_statistics_df.columns
    ]
    columns_to_subset += ["bird", "unit", "categoricality"]
    stat_df = unit_statistics_df[columns_to_subset].melt(id_vars=["bird", "unit"])
    stat_df.columns = ["bird", "unit", "interp", col]
    stat_df["interp"] = [i[-2:] for i in stat_df.interp.values]
    stat_dfs.append(stat_df)

100%|██████████| 34/34 [00:01<00:00, 21.09it/s]


In [15]:
stat_df = reduce(lambda x, y: pd.merge(x, y, on = ['bird', 'unit', 'interp']), stat_dfs)

In [16]:
stat_df = pd.merge(stat_df, unit_statistics_df_cat[
    [
        "bird",
        "unit",
        "n_trials",
        "n_cued_trials",
        "n_incorrect_trials",
        "n_correct_trials",
        "n_reward_trials",
        "n_passive_trials",
        "n_active_trials",
        "n_spikes",
        "categorical_unit",
        "categoricality",
        
    ]
], on = ['bird', 'unit'])

for interped_column in tqdm(interped_columns_unique):
    prop_nan = np.array(
        [
            np.mean(np.isnan(i)) if type(i) == np.ndarray else 1
            for i in tqdm(stat_df[interped_column].values, leave=False, desc=interped_column)
        ]
    )
    stat_df["{}_prop_nan".format(interped_column)] = prop_nan

nm_inflection_correlation:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                                     
nm_inflection_euclidean:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                                   
nm_max_correlation:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                              
nm_max_euclidean:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                            
nm_min_correlation:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                              
nm_min_euclidean:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                            
nm_r2_correlation:   0%|          | 0/114818 [00:00<?, ?it/s]
                                                             
nm_r2_euclidean:   0%|          | 0/114818 [00:00<?, ?it/s]
                                            

In [17]:
stat_df[:3]

,bird,unit,interp,ips,nm_inflection_correlation,nm_inflection_cosine,nm_inflection_euclidean,nm_inflection_manhattan,nm_max_correlation,nm_max_cosine,nm_max_euclidean,nm_max_manhattan,nm_min_correlation,nm_min_cosine,nm_min_euclidean,nm_min_manhattan,nm_r2_correlation,nm_r2_cosine,nm_r2_euclidean,nm_r2_manhattan,nm_range_correlation,nm_range_cosine,nm_range_euclidean,nm_range_manhattan,nm_scaled_slope_correlation,nm_scaled_slope_cosine,nm_scaled_slope_euclidean,nm_scaled_slope_manhattan,nm_slope_correlation,nm_slope_cosine,nm_slope_euclidean,nm_slope_manhattan,rv,sm_correlation,sm_cosine,sm_euclidean,sm_manhattan,n_trials,n_cued_trials,n_incorrect_trials,n_correct_trials,n_reward_trials,n_passive_trials,n_active_trials,n_spikes,categorical_unit,categoricality,ips_prop_nan,nm_inflection_correlation_prop_nan,nm_inflection_cosine_prop_nan,nm_inflection_euclidean_prop_nan,nm_inflection_manhattan_prop_nan,nm_max_correlation_prop_nan,nm_max_cosine_prop_nan,nm_max_euclidean_prop_nan,nm_max_manhattan_prop_nan,nm_min_correlation_prop_nan,nm_min_cosine_prop_nan,nm_min_euclidean_prop_nan,nm_min_manhattan_prop_nan,nm_r2_correlation_prop_nan,nm_r2_cosine_prop_nan,nm_r2_euclidean_prop_nan,nm_r2_manhattan_prop_nan,nm_range_correlation_prop_nan,nm_range_cosine_prop_nan,nm_range_euclidean_prop_nan,nm_range_manhattan_prop_nan,nm_scaled_slope_correlation_prop_nan,nm_scaled_slope_cosine_prop_nan,nm_scaled_slope_euclidean_prop_nan,nm_scaled_slope_manhattan_prop_nan,nm_slope_correlation_prop_nan,nm_slope_cosine_prop_nan,nm_slope_euclidean_prop_nan,nm_slope_manhattan_prop_nan,rv_prop_nan,sm_correlation_prop_nan,sm_cosine_prop_nan,sm_euclidean_prop_nan,sm_manhattan_prop_nan
0,B1597,9474,AE,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7.529137,7.12514,7.152187,7.132204,0.628102,0.582507,0.509234,0.501483,0.212181,0.314192,0.449464,0.489202,0.597437,0.822434,0.847986,0.800823,0.415921,0.268315,0.05977,0.012281,138.528892,372.696333,1673.068227,8142.778156,57.617025,100.0,100.0,100.0,"[[0.007944877, 0.012419521, 0.016634403, 0.019...","[[1.0, 0.960486, 0.9149142, 0.9587056, 0.95678...","[[1.0, 0.9633575, 0.9301829, 0.9659082, 0.9627...","[[1.0, 0.9145882, 0.90759414, 0.86016595, 0.88...","[[1.0, 0.9858349, 0.9847991, 0.97092766, 0.979...",5612,9,0,0,0,5600,12,2358,True,0.109401,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0
1,B1597,9474,AF,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7.029179,7.060066,13.0,8.524363,0.490924,0.493951,0.531064,0.501791,0.461844,0.46966,0.480647,0.495942,0.543072,0.552026,0.848617,0.729715,0.02908,0.024292,0.050417,0.005849,3438.7674,4115.76963,56.209475,792.884808,100.0,99.97865,2.833932,4.637577,"[[0.016027803, 0.024336038, 0.031571742, 0.035...","[[1.0, 0.9600967, 0.95765334, 0.9563544, 0.969...","[[1.0, 0.96551675, 0.9567401, 0.96169275, 0.96...","[[1.0, 0.88105357, 0.885755, 0.70282775, 0.911...","[[1.0, 0.9632765, 0.97379357, 0.91801924, 0.98...",5612,9,0,0,0,5600,12,2358,True,0.109401,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0
2,B1597,9474,AG,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",8.204379,8.609525,7.551757,7.541294,0.518823,0.514424,0.515437,0.503097,0.497939,0.498819,0.49274,0.497738,0.362538,0.378482,0.553078,0.333736,0.020884,0.015605,0.022697,0.005359,168.034211,221.892153,4400.962433,18661.666382,3.509229,3.462694,99.890707,100.0,"[[0.025025528, 0.033703506, 0.03998994, 0.0414...","[[1.0, 0.8948454, 0.89415073, 0.5736817, 0.961...","[[1.0, 0.9090164, 0.9004372, 0.6424036, 0.9646...","[[1.0, 0.8216941, 0.7911576, 0.73795325, 0.786...","[[1.0, 0.9635847, 0.95689726, 0.92169976, 0.94...",5612,9,0,0,0,5600,12,2358,True,0.109401,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0


In [18]:
slope_df = stat_df[[
    'bird',
    'unit',
    'interp',
    'nm_r2_cosine',
    'nm_min_cosine',
    'nm_max_cosine',
    'nm_slope_cosine',
    'nm_inflection_cosine',
    'sm_cosine_prop_nan'
    
]].rename({
    'nm_r2_cosine':'r2',
    'sm_cosine_prop_nan':'sm_pct_nan',
    'nm_slope_cosine':'slope',
    'nm_max_cosine':'_max',
    'nm_min_cosine':'_min',
    'nm_inflection_cosine':'infleciton',
}, axis='columns')
slope_df['range'] = slope_df['_max']  - slope_df['_min'] 
slope_df['scaled_neurometric_slope'] = (slope_df['slope']  * slope_df['range'] ).astype(float)
slope_df["log_scaled_neurometric_slope"] = np.log( 1+
    slope_df.scaled_neurometric_slope.values.astype(float)
)

In [19]:
slope_df[:3]

,bird,unit,interp,r2,_min,_max,slope,infleciton,sm_pct_nan,range,scaled_neurometric_slope,log_scaled_neurometric_slope
0,B1597,9474,AE,0.822434,0.314192,0.582507,100.0,7.12514,0.0,0.268315,26.831496,3.326168
1,B1597,9474,AF,0.552026,0.46966,0.493951,99.97865,7.060066,0.0,0.024292,2.428642,1.232164
2,B1597,9474,AG,0.378482,0.498819,0.514424,3.462694,8.609525,0.0,0.015605,0.054036,0.052627


In [20]:
# remove bad fits
print(len(slope_df))
slope_df = slope_df[slope_df.sm_pct_nan < 0.01]
print(len(slope_df))
slope_df = slope_df[slope_df.slope.values.astype(float) < 99]
print(len(slope_df))

114818
65273
52383


### load behavior

In [21]:
fit_df_interp = pd.read_pickle(DATA_DIR/'behavior'/'fit_df_interp.pickle')

In [22]:
fit_df_interp[:3]

,bird,cue,interp,n_points,_min,_max,_inflection,_slope,model,y_interp,r_squared
0,B1174,NC,AE,14538,0.112693,0.903653,64.036638,14.918588,<lmfit.minimizer.MinimizerResult object at 0x7...,"[0.11269340812421869, 0.11269340812421869, 0.1...",0.538664
1,B1174,CL0,AE,6876,0.099140,0.880312,64.357732,27.363792,<lmfit.minimizer.MinimizerResult object at 0x7...,"[0.09913953431260725, 0.09913953431260725, 0.0...",0.533202
2,B1174,CN,AE,1606,0.131581,0.884359,64.226503,24.816837,<lmfit.minimizer.MinimizerResult object at 0x7...,"[0.1315810264285665, 0.1315810264285665, 0.131...",0.519944


In [23]:
fit_df_interp["scaled_psychometric_slope"] = (
    fit_df_interp._slope * (fit_df_interp._max - fit_df_interp._min)
).values.astype(float)

In [24]:
psychometric_slope_df = fit_df_interp[fit_df_interp.cue == 'NC'][['bird', 'interp', '_slope', 'scaled_psychometric_slope']]

In [25]:
psychometric_slope_df[:3]

,bird,interp,_slope,scaled_psychometric_slope
0,B1174,AE,14.918588,11.799994
6,B1174,BF,18.848140,14.032384
12,B1174,AF,6.992853,6.184746


In [26]:
neurometric_psychometric_slope_df = slope_df.merge(
    psychometric_slope_df, on=["bird", "interp"]
)
neurometric_psychometric_slope_df = neurometric_psychometric_slope_df.rename(
    columns={"_slope": "psychometric_slope", "slope": "neurometric_slope"}
)
neurometric_psychometric_slope_df[:3]

,bird,unit,interp,r2,_min,_max,neurometric_slope,infleciton,sm_pct_nan,range,scaled_neurometric_slope,log_scaled_neurometric_slope,psychometric_slope,scaled_psychometric_slope
0,B1597,9474,AG,0.378482,0.498819,0.514424,3.462694,8.609525,0.0,0.015605,0.054036,0.052627,14.223974,11.699674
1,B1597,5028,AG,0.808406,0.498564,0.50668,3.055003,13.0,0.0,0.008115,0.024792,0.024490,14.223974,11.699674
2,B1597,3411,AG,0.803053,0.197903,0.934619,4.941744,9.990566,0.0,0.736717,3.640664,1.534858,14.223974,11.699674


In [27]:
neurometric_psychometric_slope_df["log_scaled_psychometric_slope"] = np.log(1+
    neurometric_psychometric_slope_df.scaled_psychometric_slope
)

In [28]:
len(neurometric_psychometric_slope_df)

41181

### Plot relationship between neurometric and psychometric

In [29]:
def get_descriptive_neurometric_psychometric(
    bird_neurometric_psychometric,
    neurometric_column="neurometric_slope",
    psychometric_column="psychometric_slope",
):
    neurometric_description = (
        bird_neurometric_psychometric[["interp", neurometric_column]]
        .groupby(["interp"])
        .describe()[[neurometric_column]][neurometric_column][["count", "mean", "std"]]
    )
    neurometric_description["sem"] = neurometric_description["std"] / np.sqrt(
        neurometric_description["count"]
    )
    neurometric_description = neurometric_description[["mean", "sem"]]
    neurometric_description.columns = ["neurometric_mean", "neurometric_sem"]

    psychometric_description = (
        bird_neurometric_psychometric[["interp", psychometric_column]]
        .groupby(["interp"])
        .describe()[[psychometric_column]][psychometric_column][["mean"]]
    )
    psychometric_description.columns = ["psychometric_mean"]
    neurometric_psychometric_mean = pd.concat(
        [psychometric_description, neurometric_description], axis=1
    )
    return neurometric_psychometric_mean

def z_score(x):
    return (x-np.mean(x))/np.std(x)

In [30]:
neurometric_psychometric_slope_df[:3]

,bird,unit,interp,r2,_min,_max,neurometric_slope,infleciton,sm_pct_nan,range,scaled_neurometric_slope,log_scaled_neurometric_slope,psychometric_slope,scaled_psychometric_slope,log_scaled_psychometric_slope
0,B1597,9474,AG,0.378482,0.498819,0.514424,3.462694,8.609525,0.0,0.015605,0.054036,0.052627,14.223974,11.699674,2.541576
1,B1597,5028,AG,0.808406,0.498564,0.50668,3.055003,13.0,0.0,0.008115,0.024792,0.024490,14.223974,11.699674,2.541576
2,B1597,3411,AG,0.803053,0.197903,0.934619,4.941744,9.990566,0.0,0.736717,3.640664,1.534858,14.223974,11.699674,2.541576


##### Z score scaled neurometric and psychometric by bird

In [31]:
neurometric_psychometric_slope_df_list = []
for bird in neurometric_psychometric_slope_df.bird.unique():
    bird_neurometric_psychometric = neurometric_psychometric_slope_df[
        neurometric_psychometric_slope_df.bird == bird
    ]
    bird_neurometric_psychometric["z_score_log_scaled_neurometric_slope"] = z_score(
        bird_neurometric_psychometric["log_scaled_neurometric_slope"]
    )
    bird_neurometric_psychometric["z_score_log_scaled_psychometric_slope"] = z_score(
        bird_neurometric_psychometric["log_scaled_psychometric_slope"]
    )
    bird_neurometric_psychometric["z_score_scaled_neurometric_slope"] = z_score(
        bird_neurometric_psychometric["scaled_neurometric_slope"]
    )
    bird_neurometric_psychometric["z_score_scaled_psychometric_slope"] = z_score(
        bird_neurometric_psychometric["scaled_psychometric_slope"]
    )
    neurometric_psychometric_slope_df_list.append(bird_neurometric_psychometric)
neurometric_psychometric_slope_df = pd.concat(neurometric_psychometric_slope_df_list)

/tmp/ipykernel_26228/416860814.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bird_neurometric_psychometric["z_score_log_scaled_neurometric_slope"] = z_score(
/tmp/ipykernel_26228/416860814.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bird_neurometric_psychometric["z_score_log_scaled_psychometric_slope"] = z_score(
/tmp/ipykernel_26228/416860814.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [32]:
neurometric_psychometric_slope_df[:3]

,bird,unit,interp,r2,_min,_max,neurometric_slope,infleciton,sm_pct_nan,range,scaled_neurometric_slope,log_scaled_neurometric_slope,psychometric_slope,scaled_psychometric_slope,log_scaled_psychometric_slope,z_score_log_scaled_neurometric_slope,z_score_log_scaled_psychometric_slope,z_score_scaled_neurometric_slope,z_score_scaled_psychometric_slope
0,B1597,9474,AG,0.378482,0.498819,0.514424,3.462694,8.609525,0.0,0.015605,0.054036,0.052627,14.223974,11.699674,2.541576,-0.892697,0.664578,-0.324122,0.588053
1,B1597,5028,AG,0.808406,0.498564,0.50668,3.055003,13.0,0.0,0.008115,0.024792,0.024490,14.223974,11.699674,2.541576,-0.951790,0.664578,-0.334893,0.588053
2,B1597,3411,AG,0.803053,0.197903,0.934619,4.941744,9.990566,0.0,0.736717,3.640664,1.534858,14.223974,11.699674,2.541576,2.220242,0.664578,0.996917,0.588053


In [33]:
df = neurometric_psychometric_slope_df[
    [
        "bird",
        "unit",
        "interp",
        "log_scaled_neurometric_slope",
        "log_scaled_psychometric_slope",
    ]
].sort_values(by=['unit', 'interp'])

In [34]:
df["bird_unit"] = ["".join([row.bird, row.unit]) for idx, row in df.iterrows()]

In [35]:
df[:3]

,bird,unit,interp,log_scaled_neurometric_slope,log_scaled_psychometric_slope,bird_unit
11855,B1276,0,AE,1.034476,2.977097,B12760
22541,B1244,0,AE,0.543070,2.814173,B12440
31660,B1593,0,AE,0.519640,2.376957,B15930


In [41]:
len(df)

41181

In [37]:
import statsmodels.formula.api as smf

In [43]:
# Fit a linear mixed-effects model
model = smf.mixedlm("log_scaled_neurometric_slope ~ log_scaled_psychometric_slope", 
                    data=df,
                    groups=df["bird_unit"],
                    missing="drop")
result = model.fit()
print(result.summary())

                  Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: log_scaled_neurometric_slope
No. Observations: 41181   Method:             REML                        
No. Groups:       6713    Scale:              0.1498                      
Min. group size:  1       Log-Likelihood:     -25523.0561                 
Max. group size:  9       Converged:          Yes                         
Mean group size:  6.1                                                     
--------------------------------------------------------------------------
                                 Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------
Intercept                        0.204    0.011 18.921 0.000  0.183  0.225
log_scaled_psychometric_slope    0.170    0.004 39.523 0.000  0.161  0.178
Group Var                        0.142    0.009                           



In [44]:
# Fit a linear mixed-effects model
model = smf.mixedlm("log_scaled_neurometric_slope ~ log_scaled_psychometric_slope", 
                    data=df,
                    groups=df["bird_unit"],
                    re_formula="1 + log_scaled_psychometric_slope",
                    missing="drop")
result = model.fit()
print(result.summary())

                       Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    log_scaled_neurometric_slope
No. Observations:     41181      Method:                REML                        
No. Groups:           6713       Scale:                 0.1457                      
Min. group size:      1          Log-Likelihood:        -25483.2827                 
Max. group size:      9          Converged:             Yes                         
Mean group size:      6.1                                                           
------------------------------------------------------------------------------------
                                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------
Intercept                                  0.202    0.011 17.980 0.000  0.180  0.224
log_scaled_psychometric_slope              0.172    0.005 37.006 0.000  0.163  0.181
Grou

In [45]:
# Fit a linear mixed-effects model
model = smf.mixedlm("log_scaled_neurometric_slope ~ log_scaled_psychometric_slope + C(interp)", 
                    data=df,
                    groups=df["bird_unit"],
                    re_formula="1 + log_scaled_psychometric_slope",
                    missing="drop")
result = model.fit()
print(result.summary())

                        Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    log_scaled_neurometric_slope
No. Observations:      41181      Method:                REML                        
No. Groups:            6713       Scale:                 0.1410                      
Min. group size:       1          Log-Likelihood:        -24804.5443                 
Max. group size:       9          Converged:             Yes                         
Mean group size:       6.1                                                           
-------------------------------------------------------------------------------------
                                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                  0.801    0.022  36.865 0.000  0.758  0.844
C(interp)[T.AF]                           -0.136    0.009 -15.669 0.000 -0.153